<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Laboratory Scenario 2 - Backpropagation and Gradient Checkpointing

In this lab scenario, you are given an implementation of a simple neural network, and your goal is to implement the backpropagation procedure for this network.  

To be more precise, the network inputs a tensor $x$ of shape `(MINI_BATCH_SIZE, 28*28)`, where each element of the batch represents a flattened grayscale image of shape `(28, 28)`.  
In exercise 1, you can assume that images in the minibatch are fed to the network one by one (as tensors of shape `(1, 28*28)` - single image and `(1, 10)` - image class).  
In exercise 2 you are asked to make the backpropagation work without this assumption, on whole mini-batches.  
In exercise 3, you will implement a technique called *gradient checkpointing*, that allows you to reduce the amount of memory used to store activations for backpropagation.

In [ ]:
from pathlib import Path
from typing import Sequence, Iterator, cast
from typing_extensions import override

import numpy as np
import PIL.Image
from numpy.typing import NDArray
from tqdm.auto import tqdm

FloatNDArray = NDArray[np.float64]

## Loading the MNIST dataset

In [ ]:
!wget --no-verbose -O mnist.npz https://s3.amazonaws.com/img-datasets/mnist.npz

In [ ]:
def load_mnist(
    path: Path = Path('mnist.npz')
) -> tuple[FloatNDArray, FloatNDArray, FloatNDArray, FloatNDArray]:
    '''
    Load the MNIST dataset (grayscale 28 x 28 images of hand-written digits).

    Returns tuple of:
    - x_train: shape (N_train, H * W), grayscale values 0..1.
    - y_train: shape (N_train, 10), one-hot-encoded label, dtype float64.
    - x_test: shape (N_test, H * W), grayscale values 0..1.
    - y_train: shape (N_test, 10), one-hot-encoded label, dtype float64.

    More: https://en.wikipedia.org/wiki/MNIST_database
    '''
    with np.load(path) as f:
        x_train, _y_train = f['x_train'], f['y_train']
        x_test, _y_test = f['x_test'], f['y_test']

    H = W = 28
    N_train = len(x_train)
    N_test = len(x_test)
    assert x_train.shape == (N_train, H, W) and _y_train.shape == (N_train,)
    assert x_test.shape == (N_test, H, W) and _y_test.shape == (N_test,)

    x_train = x_train.reshape(N_train, H * W) / 255.0
    x_test = x_test.reshape(N_test, H * W) / 255.0

    y_train = np.zeros((N_train, 10), dtype=np.float64)
    y_train[np.arange(N_train), _y_train] = 1

    y_test = np.zeros((N_test, 10))
    y_test[np.arange(N_test), _y_test] = 1

    return x_train, y_train, x_test, y_test


x_train, y_train, x_test, y_test = load_mnist()

In [ ]:
def to_pillow_image(x: FloatNDArray, scale: int = 10) -> PIL.Image.Image:
    '''Convert example of shape (28 * 28,) and values 0..1 to Pillow image.'''
    H = W = 28
    assert x.shape == (H * W,) and 0 <= x.min() <= x.max() <= 1
    x_int = (x * 255).astype(np.uint8).reshape(H, W)
    img = PIL.Image.fromarray(x_int, mode='L')
    return img.resize((H * scale, W * scale), PIL.Image.Resampling.NEAREST)


display(to_pillow_image(x_train[0]))
print(y_train[0])

## Exercise 1

In this exercise, your task is to fill in the gaps in this code by implementing the backpropagation algorithm.
Once done, you can run the network on the MNIST example and see how it performs.  
Feel free to play with the parameters. Your model should achieve 90%+ accuracy after a few epochs.  

Before you start you should note a few things:
+ `backprop` - is the function that you need to implement
+ `learning_step` - calls `backprop` to get the gradients for network parameters
+ The derivative of the loss is already computed by `cost_derivative`.
+ Your goal is to compute $\frac{d L\left(\text{model}(x), y\right)}{d p}$ for each parameter $p$ of the network


In [ ]:
def sigmoid(z: FloatNDArray) -> FloatNDArray:
    return 1.0 / (1.0 + np.exp(-z))


def sigmoid_prime(z: FloatNDArray) -> FloatNDArray:
    '''Derivative of the sigmoid.'''
    return sigmoid(z) * (1 - sigmoid(z))

In [ ]:
class Network:
    '''
    Multi-Layer Perceptron.

    A simple neural network with fully-connected layers and sigmoid activations.
    '''

    def __init__(self, sizes: Sequence[int] = (784, 30, 10)):
        '''
        Args:
        - sizes: sequence of layer widths [N^0, ... , N^last]
          These are lengths of activation vectors, where:
          - N^0 is input size: 784.
          - N^last is the number of classes into which we can classify each input: 10.
        '''
        self.sizes = list(sizes)

        # We initialize weights and biases with random normal distribution.

        # List of len(sizes) - 1 vectors of shape (N^1), (N^2), ..., (N^last).
        self.biases = [np.random.randn(n) for n in sizes[1:]]

        # List of len(sizes) - 1 matrices of shape (N^i, N^{i-1}).
        # Weights are indexed by target node first.
        self.weights = [
            np.random.randn(n_out, n_in) / np.sqrt(n_in)
            for n_in, n_out in zip(sizes[:-1], sizes[1:], strict=True)
        ]

    def feedforward(self, x: FloatNDArray) -> FloatNDArray:
        '''
        Run the network on a single case of shape (N^0,).

        Returns last layer activations, shape (N^last,), values 0..1.
        '''
        g = x
        for w, b in zip(self.weights, self.biases, strict=True):
            f = w @ g + b
            g = sigmoid(f)
            pass
        return g

    def learning_step(
        self, x_mini_batch: FloatNDArray, y_mini_batch: FloatNDArray, learning_rate: float
    ) -> None:
        '''
        Update network parameters with a single mini-batch step of backpropagation and gradient descent.

        Args:
        - x_mini_batch: shape (B, N^0) where B is the mini-batch size.
        - y_mini_batch: shape (B, N^last).
        - learning_rate.
        '''
        # Accumulate gradients by running backprop one dataitem at a time (without vectorization).
        grads_w = [np.zeros(w.shape) for w in self.weights]
        grads_b = [np.zeros(b.shape) for b in self.biases]
        for x, y in zip(x_mini_batch, y_mini_batch):
            item_grads_w, item_grads_b = self.backprop(x, y)
            for i in range(len(grads_w)):
                grads_w[i] = grads_w[i] + item_grads_w[i]
            for i in range(len(grads_b)):
                grads_b[i] = grads_b[i] + item_grads_b[i]

        # Gradient descent step.
        self.weights = [
            w - grad_w * (learning_rate / len(x_mini_batch))
            for w, grad_w in zip(self.weights, grads_w, strict=True)
        ]
        self.biases = [
            b - grad_b * (learning_rate / len(x_mini_batch))
            for b, grad_b in zip(self.biases, grads_b, strict=True)
        ]

    def backprop(
        self, x: FloatNDArray, y: FloatNDArray
    ) -> tuple[list[FloatNDArray], list[FloatNDArray]]:
        '''
        Backpropagation for a single input (not vectorized).

        Args:
        - x: input features, shape (N^0).
        - y: target label (one-hot encoded), shape (N^last).

        Returns (grads_w, grads_b), where:
        - grads_w is a list of gradients over weights (shape (N^i, N^{i-1})), for each layer.
        - grads_b is a list of gradients over biases (shape (N^i)), for each layer.
        '''

        # Go forward, remembering all activations.
        # Pre-activations function, layer by layer, shapes (N^1), ..., (N^last).
        fs: list[FloatNDArray] = []
        # Activations (including inputs to the first layer), shapes (N^0), (N^1), ..., (N^last).
        gs: list[FloatNDArray] = [x]

        # TODO forward pass.

        g = x
        for w, b in zip(self.weights, self.biases, strict=True):
            f = w @ g + b
            g = sigmoid(f)

            fs.append(f)
            gs.append(g)


        assert [f.shape for f in fs] == [(n,) for n in self.sizes[1:]], f'Shape mismatch: {[f.shape for f in fs]} vs {self.sizes[1:]}'
        assert [g.shape for g in gs] == [(n,) for n in self.sizes], f'Shape mismatch: {[g.shape for g in gs]} vs {self.sizes}'

        # Now go backward from the final cost applying backpropagation.
        grad_g = self.cost_derivative(gs[-1], y)  # shape initially (N^last), then layer by layer.


        # TODO backward pass. May be useful:
        # - reversed(list(...)) can be used to iterate in reverse.
        # - list.reverse() reverses a list in-place.
        # - np.outer() computes an outer product (a_{i,j} = b_i c_j) very fast.

        grads_w = []
        grads_b = []

        prev_g = gs[-1]
        for f, g, w in zip(reversed(fs), reversed(gs[:-1]), reversed(self.weights)):
            grad_f = grad_g*prev_g*(1-prev_g)
            grad_g = w.T @ grad_f
            grads_w.append(np.outer(grad_f, g))
            grads_b.append(grad_f)
            prev_g = g

        grads_w.reverse()
        grads_b.reverse()


        # Now grads_w should have shapes (N^1, N^0), ..., (N^last, N^{last-1}).
        # Now grads_b should have shapes (N^1), ..., (N^last).
        for grad_b, b in zip(grads_b, self.biases, strict=True):
            assert grad_b.shape == b.shape, f'Shape mismatch: {grad_b.shape=} but {b.shape=}'
        for grad_w, w in zip(grads_w, self.weights, strict=True):
            assert grad_w.shape == w.shape, f'Shape mismatch: {grad_w.shape=} but {w.shape=}'

        return grads_w, grads_b

    def cost_derivative(self, g: FloatNDArray, y: FloatNDArray) -> FloatNDArray:
        '''
        Gradient of loss (MSE) over output activations, for a single sample.

        Args:
        - g: output activations, shape (N^last).
        - y: target values (one-hot encoded labels), shape (N^last).

        Returns gradients, shape (N^last).
        '''
        assert g.shape == y.shape, f'Shape mismatch: {g.shape=} but {y.shape=}'
        N_last, = g.shape
        return (2 / N_last) * (g - y.astype(np.float64))

    def evaluate(
        self, x_test_data: FloatNDArray, y_test_data: FloatNDArray
    ) -> np.float64:
        '''
        Compute accuracy: the ratio of correct answers for test_data.

        Args (here B is the number of test dataitems):
        - x_test_data: shape (B, N^0).
        - y_test_data: shape (B, N^last).
        '''
        test_results: list[bool] = []
        for x, y in zip(x_test_data, y_test_data):
            output_label: np.int64 = np.argmax(self.feedforward(x))
            target_label: np.int64 = np.argmax(y)
            test_results.append(output_label == target_label)

        return np.mean(test_results)

    def SGD(
        self,
        training_data: tuple[FloatNDArray, FloatNDArray],
        test_data: tuple[FloatNDArray, FloatNDArray] | None = None,
        epochs: int = 2,
        mini_batch_size: int = 100,
        learning_rate: float = 1.0,
    ) -> None:
        x_train, y_train = training_data
        for epoch in tqdm(range(epochs)):
            for i in range(x_train.shape[0] // mini_batch_size):
                i_begin = i * mini_batch_size
                i_end = (i + 1) * mini_batch_size
                self.learning_step(x_train[i_begin:i_end], y_train[i_begin:i_end], learning_rate)
            if test_data:
                x_test, y_test = test_data
                accuracy = self.evaluate(x_test, y_test)
                tqdm.write(f'Epoch: {epoch}, Accuracy: {accuracy * 100:.2f} %')
            else:
                tqdm.write(f'Epoch: {epoch}')

In [ ]:
%%time
# Smaller test on part of the dataset.
# The unvectorized version should take about 2s per epoch
# and achieve accuracy ~75% or more (for most executions).
network = Network([784, 30, 10])
network.SGD(
    (x_train[:3000], y_train[:3000]),
    test_data=(x_test[:3000], y_test[:3000]),
    epochs=5,
    mini_batch_size=100,
    learning_rate=10.0
)

In [32]:
%%time
# Full test.
# The unvectorized version should take about 20s per epoch
# and achieve accuracy ~90% or more.
network = Network([784, 30, 10])
network.SGD(
    (x_train, y_train),
    test_data=(x_test, y_test),
    epochs=10,
    mini_batch_size=100,
    learning_rate=5.0
)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0, Accuracy: 88.60 %


KeyboardInterrupt: 

## Exercise 2

Implement a "fully vectorized" version, i.e. one using matrix operations instead of going over examples one by one within a minibatch.


In [33]:
class NetworkVectorized:
    '''Multi-Layer Perceptron with vectorized (batched) methods.'''

    def __init__(self, sizes: Sequence[int] = (784, 30, 10)):
        '''
        Args:
        - sizes: sequence of layer widths [N^0, ... , N^last]
          These are lengths of activation vectors, where:
          - N^0 is input size: 784.
          - N^last is the number of classes into which we can classify each input: 10.
        '''
        self.sizes = list(sizes)

        # We initialize weights and biases with random normal distribution.

        # List of len(sizes) - 1 vectors of shape (N^1), (N^2), ..., (N^last)
        self.biases = [np.random.randn(n) for n in sizes[1:]]

        # List of len(sizes) - 1 matrices of shape (N^i, N^{i-1}).
        # Weights are indexed by target node first.
        self.weights = [
            np.random.randn(n_out, n_in) / np.sqrt(n_in)
            for n_in, n_out in zip(sizes[:-1], sizes[1:], strict=True)
        ]

    def feedforward(self, x: FloatNDArray) -> FloatNDArray:
        '''
        Run the network on a batch of cases of shape (B, N^0), values 0..1.

        Returns last layer activations, shape (B, N^last), values 0..1.
        '''
        g = x
        for w, b in zip(self.weights, self.biases, strict=True):
            f = g @ w.T + b # ((B, N^i) @ (N^{i-1}, N^{i}).T) <- ((N^{i-1}, N^{i}) @ (B, N^i).T).T
            g = sigmoid(f)
        return g

    def learning_step(self, x_mini_batch: FloatNDArray, y_mini_batch: FloatNDArray, learning_rate: float) -> None:
        '''
        Update network parameters with a single mini-batch step of backpropagation and gradient descent.

        Args:
        - x_mini_batch: shape (B, N^0) where B is mini_batch_size.
        - y_mini_batch: shape (B, N^last).
        - learning_rate.
        '''
        grads_w, grads_b = self.backprop_vectorized(x_mini_batch, y_mini_batch)

        # Gradient descent step.
        self.weights = [
            w - learning_rate * grad_w
            for w, grad_w in zip(self.weights, grads_w, strict=True)
        ]
        self.biases = [
            b - learning_rate * grad_b
            for b, grad_b in zip(self.biases, grads_b, strict=True)
        ]

    def backprop_vectorized(
        self, x: FloatNDArray, y: FloatNDArray
    ) -> tuple[list[FloatNDArray], list[FloatNDArray]]:
        '''Backpropagation for a mini-batch (vectorized).

        Args:
        - x: input, shape (B, N^0)
        - y: target label (one-hot encoded), shape (B, N^last)

        Returns (grads_w, grads_b), where:
        - grads_w: list of gradients over weights (shape (N^i, N^{i-1})), for each layer.
        - grads_b: list of gradients over biases (shape (N^i)), for each layer.
        '''
        B, N0 = x.shape
        assert N0 == self.sizes[0]

        # Go forward, remembering all activations.

        # Values after activation function (including inputs to the first layer),
        # shapes (B, N^0), (B, N^1), ..., (B, N^last).
        gs: list[FloatNDArray] = [x]
        g = x
        for w, b in zip(self.weights, self.biases, strict=True):
            f = g @ w.T + b
            g = sigmoid(f)
            gs.append(g)


        # TODO

        assert [g.shape for g in gs] == [(B, n) for n in self.sizes], \
            f'Shape mismatch: {[g.shape for g in gs]} vs {[(B, n) for n in self.sizes]}'

        # Now go backward from the final cost applying backpropagation.
        grad_g = self.cost_derivative(gs[-1], y)  # shape initially (B, N^last), then layer by layer.


        # TODO
        # Vectorized backward pass over a mini-batch
        # Inputs available:
        # - Gs: list of activations per layer, shapes [(B, N^0), (B, N^1), ..., (B, N^last)]
        # - weights: list of W_i with shapes [(N^1, N^0), ..., (N^last, N^{last-1})]
        # - grad_G: dC/dG at output layer, shape (B, N^last)

        grads_w = []
        grads_b = []

        g_curr = gs[-1]

        for g_prev, w in zip(reversed(gs[:-1]), reversed(self.weights)):
            grad_f = grad_g * g_curr * (1.0 - g_curr)   # (B, N^i)

            grad_w = grad_f.T @ g_prev                  # (N^i, N^{i-1})
            grad_b = grad_f.sum(axis=0)                 # (N^i,)

            grads_w.append(grad_w)
            grads_b.append(grad_b)

            grad_g = grad_f @ w                         # (B, N^{i-1})

            g_curr = g_prev

        grads_w.reverse()
        grads_b.reverse()


        # Now grads_w should have shapes (N^1, N^0), ..., (N^last, N^{last-1}).
        # Now grads_b should have shapes (N^1,) ..., (N^last,).
        for grad_b, b in zip(grads_b, self.biases, strict=True):
            assert grad_b.shape == b.shape, f'Shape mismatch: {grad_b.shape=} but {b.shape=}'
        for grad_w, w in zip(grads_w, self.weights, strict=True):
            assert grad_w.shape == w.shape, f'Shape mismatch: {grad_w.shape=} but {w.shape=}'

        return grads_w, grads_b

    def cost_derivative(self, g: FloatNDArray, y: FloatNDArray) -> FloatNDArray:
        '''
        Gradient of loss (MSE) over output activations.

        Args:
        - g: output activations, shape (B, N^last).
        - y: target values (one-hot encoded labels), shape (B, N^last).

        Returns gradients, shape (B, N^last).
        '''
        assert g.shape == y.shape, f'Shape mismatch: {g.shape=} but {y.shape=}'
        B, N_last = g.shape
        return (2 / (B * N_last)) * (g - y.astype(np.float64))

    def evaluate(self, x_test_data: FloatNDArray, y_test_data: FloatNDArray) -> np.float64:
        '''
        Compute accuracy: the ratio of correct answers for test_data.

        Args:
        - x_test_data: shape (B, N^0).
        - y_test_data: shape (B, N^last).
        '''
        predictions = np.argmax(self.feedforward(x_test_data), axis=1)
        targets = np.argmax(y_test_data, axis=1)
        return np.mean(predictions == targets)

    def SGD(
        self,
        training_data: tuple[FloatNDArray, FloatNDArray],
        test_data: tuple[FloatNDArray, FloatNDArray] | None = None,
        epochs: int = 2,
        mini_batch_size: int = 100,
        learning_rate: float = 1.0
    ) -> None:
        x_train, y_train = training_data
        for epoch in tqdm(range(epochs)):
            for i in range(x_train.shape[0] // mini_batch_size):
                i_begin = i * mini_batch_size
                i_end = (i + 1) * mini_batch_size
                self.learning_step(x_train[i_begin:i_end], y_train[i_begin:i_end], learning_rate)
            if test_data:
                x_test, y_test = test_data
                accuracy = self.evaluate(x_test, y_test)
                print(f'Epoch: {epoch}, Accuracy: {accuracy * 100:.2f} %')
            else:
                print(f'Epoch: {epoch}')

In [34]:
%%time
# The vectorized version takes about ~1s per epoch.
network = NetworkVectorized([784, 30, 10])
network.SGD(
    (x_train, y_train),
    test_data=(x_test, y_test),
    epochs=50,
    mini_batch_size=100,
    learning_rate=5.0
)

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 0, Accuracy: 88.99 %
Epoch: 1, Accuracy: 91.13 %
Epoch: 2, Accuracy: 92.06 %
Epoch: 3, Accuracy: 92.70 %


KeyboardInterrupt: 

# Exercise 3 (optional)

The standard backpropagation method requires memorization of all outputs of all layers computed during the forward pass, for use in the backward pass, which can take much of precious GPU memory.
Instead of doing that, one can checkpoint (memorize) only activations of a select few layers and then recompute the rest as they are needed (redoing the forward pass between checkpoints).  
Your task is to complete the code below to implement backpropagation with checkpoints.

In [63]:
class NetworkWithCheckpoints(NetworkVectorized):
    '''Multi-Layer Perceptron with gradient checkpointing.'''

    def __init__(self, sizes: Sequence[int] = (784, 30, 10), checkpoints: Sequence[int] = (1,)):
        '''
        Args:
        - sizes: sequence of layer widths [N^0, ... , N^last]
          These are lengths of activation vectors, where:
          - N^0 is input size: 784.
          - N^last is the number of classes into which we can classify each input: 10.
        - checkpoints: Indices of layers whose activations we want to checkpoint
          (between 1 and last inclusive (last=len(sizes) - 1).
        '''
        super().__init__(sizes)
        last_layer_id = len(sizes) - 1
        # Always store the input and last activations.
        self.checkpoints = sorted(set(checkpoints) | {0, last_layer_id})

    def backprop_generator(
        self, x: FloatNDArray, y: FloatNDArray
    ) -> Iterator[tuple[FloatNDArray, FloatNDArray]]:
        '''
        Backpropagation for a mini-batch (vectorized).

        Args:
        - x: input, shape (B, N^0)
        - y: target label (one-hot encoded), shape (B, N^last)

        Yields (grad_w, grad_b) for each layer from i=last down to i=1, where:
        - grads_w: shape (N^i, N^{i-1})).
        - grads_b: shape (N^i)).
        '''
        B, N0 = x.shape
        assert N0 == self.sizes[0]

        # Go forward, remembering only some activations and no pre-activations.

        # Values after activation function (including inputs to the first layer),
        # shapes (B, N^0), (B, N^1), ..., (B, N^last);
        # layers that are not checkpointed get None instead.
        gs: list[FloatNDArray | None] = [x]

        # TODO forward pass.
        g = x
        gs: list[FloatNDArray] = [x]
        checkpoint_num = 1
        for i, wb in enumerate(zip(self.weights, self.biases, strict=True)):
            w, b = wb
            f = g @ w.T + b
            g = sigmoid(f)
            if i+1 == self.checkpoints[checkpoint_num]:
                gs.append(g)
                checkpoint_num = min(len(self.checkpoints)-1, checkpoint_num+1)

        # Now go backward from the final cost applying backpropagation.
        grad_g = self.cost_derivative(g, y)  # shape initially (B, N^last), then layer by layer.

        # TODO backward pass.
        # for each interval between consecutive checkpoints:
            # Forward pass to restore intermediate activations.
            # for all layers in interval:
            # ...

            # Now g and grad_g have shape (B, N^{end_i}).
            #assert g.shape == grad_g.shape == (B, self.sizes[end_i])

            # Backward pass between the checkpoints.
            # for all layers in the interval, backwards:
            #   # Compute grad_w, grad_b.
            #   # Yield them as we go, instead of returning a list.
            #   # https://docs.python.org/3/tutorial/classes.html#generators
            #   yield (grad_w, grad_b)

            # Now grad_g has shape (B, N^{start_i}).
            #assert grad_g.shape == (B, self.sizes[start_i])

        prev_checkpoint = self.checkpoints[-1]

        for checkp_idx, checkpoint in zip([i for i in range(len(gs)-2, -1, -1)], reversed(self.checkpoints[:-1])):
            g = gs[checkp_idx]
            gs_temp = [g]

            for w, b in zip(self.weights[checkpoint:prev_checkpoint], self.biases[checkpoint:prev_checkpoint]):
                f = g @ w.T + b
                g = sigmoid(f)
                gs_temp.append(g)

            assert g.shape == grad_g.shape == (B, self.sizes[prev_checkpoint])

            g_curr = gs[checkp_idx+1]
            for g_prev, w in zip(reversed(gs_temp[:-1]), reversed(self.weights[checkpoint:prev_checkpoint])):
                grad_f = grad_g * g_curr * (1.0 - g_curr)   # (B, N^i)

                grad_w = grad_f.T @ g_prev                  # (N^i, N^{i-1})
                grad_b = grad_f.sum(axis=0)                 # (N^i,)

                yield grad_w, grad_b

                grad_g = grad_f @ w                         # (B, N^{i-1})

                g_curr = g_prev

            assert grad_g.shape == (B, self.sizes[checkpoint])
            prev_checkpoint = checkpoint




    @override
    def learning_step(self, x_mini_batch: FloatNDArray, y_mini_batch: FloatNDArray, learning_rate: float) -> None:
        '''
        Update network parameters with a single mini-batch step of backpropagation and gradient descent.

        Args:
        - x_mini_batch: shape (B, N^0) where B is mini_batch_size.
        - y_mini_batch: shape (B, N^last).
        - learning_rate.
        '''
        indices = range(len(self.sizes) - 1, 0, -1)  # From `last` down to 1 inclusive.
        for i, (grad_w, grad_b) in zip(indices, self.backprop_generator(x_mini_batch, y_mini_batch), strict=True):
            self.weights[i - 1] = self.weights[i - 1] - learning_rate * grad_w
            self.biases[i - 1] = self.biases[i - 1] - learning_rate * grad_b


In [64]:
%%time
# Compare on the same seed, results should be exactly the same.

np.random.seed(42)
network = NetworkWithCheckpoints([784, 20, 15, 13, 10], checkpoints=[2])
network.SGD(
    (x_train, y_train),
    test_data=(x_test, y_test),
    epochs=1,
    mini_batch_size=100,
    learning_rate=50.
)

np.random.seed(42)
network = NetworkVectorized([784, 20, 15, 13, 10])
network.SGD(
    (x_train, y_train),
    test_data=(x_test, y_test),
    epochs=1,
    mini_batch_size=100,
    learning_rate=50.
)

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 0, Accuracy: 81.38 %


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 0, Accuracy: 81.38 %
CPU times: user 3.38 s, sys: 10.7 ms, total: 3.4 s
Wall time: 1.79 s


 Memory usage can be checked by installing `memory_profiler` and using `%%memit` (instead of `%%time`),
 but it will be very hard to see a difference from checkpointing for an MLP.

# JAX Playground (Optional)
JAX is a framework that allows the creation of neural networks with numpy-like syntax.  
In this course, we will use Pytorch instead of JAX, but for this lab scenario, JAX can help us test our gradient computation implementation.  
Let's give it a try  

In [37]:
!pip3 install jax

In [38]:
import jax
import jax.numpy as jnp
from textwrap import dedent


def jax_sigmoid(z: jax.Array) -> jax.Array:
    return 1.0 / (1.0 + jnp.exp(-z))


def jax_sigmoid_prime(z: jax.Array) -> jax.Array:
    return sigmoid(z) * (1 - sigmoid(z))


# Define a jax function.
# We emphasize that this is a function, not a jax procedure,
# and in fact there are more requirements for writing good jax code,
# but this is just an example.
# (see https://jax.readthedocs.io/en/latest/tutorials.html)
def jax_forward(x: jax.Array, w: jax.Array, b: jax.Array) -> jax.Array:
    # f = TODO
    g = jax_sigmoid(f)
    loss = g.sum()  # Just a dummy loss for simplicity.
    return loss, g


# This will calculate gradient for first, second, and third argument.
# has_aux tells that in addition to loss our function returns something more.
auto_backward = jax.value_and_grad(fun=jax_forward, argnums=[0, 1, 2], has_aux=True)


def manual_backward(x: jax.Array, w: jax.Array, b: jax.Array) -> jax.Array:
    # f = TODO
    grad_g = jnp.ones_like(f)  # Grad of the dummy loss over activations g.
    # grad_f = TODO
    # grad_b = TODO
    # grad_w = TODO
    # grad_x = TODO
    return grad_x, grad_w, grad_b


def example():
    B, N0, N1 = 3, 5, 7

    key = jax.random.key(42)
    key, subkey = jax.random.split(key)
    w = jax.random.normal(subkey, (N1, N0))
    key, subkey = jax.random.split(key)
    b = jax.random.normal(subkey, (N1,))
    x = jnp.arange(N0, dtype=w.dtype).reshape(1, N0) * jnp.ones((B, N0))

    (loss, res), (jax_dx, jax_dw, jax_db) = auto_backward(x, w, b)
    dx, dw, db = manual_backward(x, w, b)

    print(dedent(f'''
        diff dx = {jnp.mean(jnp.abs(jax_dx - dx))}
        diff dw = {jnp.mean(jnp.abs(jax_dw - dw))}
        diff db = {jnp.mean(jnp.abs(jax_db - db))}
        dtype={dx.dtype} (eps={np.finfo(dx.dtype).eps})
    ''').strip())


example()

NameError: name 'f' is not defined